**Imports & Setup:**
---------------------

In [ ]:
import sys
import cv2
import pandas as pd
import gc
import os
import time
from tqdm import tqdm_notebook as tqdm
import random
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import resample
from sklearn.model_selection import train_test_split

In [ ]:
import torch
import torchvision
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from torch.utils.data import *
from torchvision import models
from torch import nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.autograd import Variable
from torch.nn.modules.distance import PairwiseDistance
import torch.utils.model_zoo as model_zoo
import torch.backends.cudnn as cudnn
from torch.utils.tensorboard import SummaryWriter

In [ ]:
# Set seed
torch.manual_seed(0)
# Set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

------------------

**Create dataframe:**
-------------
* Combine all mask to obtain a single entry per image containing all the masks
* **Assign a class based on inverse frequency:** If an image contains the least frequent class, assign this class, continue with the second least frequent class...
* Undersample the data based on least frequent class

In [ ]:
train_df = pd.read_csv('../input/severstal-steel-defect-detection/train.csv')
train_df['ImageId'] = train_df['ImageId_ClassId'].apply(lambda x: x.split('_')[0])
train_df['ClassId'] = train_df['ImageId_ClassId'].apply(lambda x: x.split('_')[1])

encoded_pixels_class_1 = []
encoded_pixels_class_2 = []
encoded_pixels_class_3 = []
encoded_pixels_class_4 = []
train_df1 = train_df[train_df['ClassId']=='1']
train_df2 = train_df[train_df['ClassId']=='2']
train_df3 = train_df[train_df['ClassId']=='3']
train_df4 = train_df[train_df['ClassId']=='4']
for image in set(train_df1['ImageId'].values.tolist()):
    encoded_pixels_class_1.append(train_df1[train_df1['ImageId']==image]['EncodedPixels'].values.tolist())
for image in set(train_df2['ImageId'].values.tolist()):
    encoded_pixels_class_2.append(train_df2[train_df2['ImageId']==image]['EncodedPixels'].values.tolist())
for image in set(train_df3['ImageId'].values.tolist()):
    encoded_pixels_class_3.append(train_df3[train_df3['ImageId']==image]['EncodedPixels'].values.tolist())
for image in set(train_df4['ImageId'].values.tolist()):
    encoded_pixels_class_4.append(train_df4[train_df4['ImageId']==image]['EncodedPixels'].values.tolist())
    
unique_images = list(set(train_df['ImageId'].values.tolist()))
data = {'ImageId': unique_images,
        'EP1': [item for sublist in encoded_pixels_class_1 for item in sublist],
        'EP2': [item for sublist in encoded_pixels_class_2 for item in sublist],
        'EP3': [item for sublist in encoded_pixels_class_3 for item in sublist],
        'EP4': [item for sublist in encoded_pixels_class_4 for item in sublist]}
train_df2 = pd.DataFrame(data)
print(train_df2.shape)
train_df2.head()

In [ ]:
class_frquencies = [
    abs(len(train_df2[train_df2['EP1'].isna()])-len(train_df2)),
    abs(len(train_df2[train_df2['EP2'].isna()])-len(train_df2)),
    abs(len(train_df2[train_df2['EP3'].isna()])-len(train_df2)),
    abs(len(train_df2[train_df2['EP4'].isna()])-len(train_df2))
]

classes = []
for i, row in train_df2.iterrows():
    if type(row['EP2']) != float:
        classes.append(2)
    elif type(row['EP4']) != float:
        classes.append(4)
    elif type(row['EP1']) != float:
        classes.append(1)
    elif type(row['EP3']) != float:
        classes.append(3)
    else:
        classes.append(0)

train_df2['class'] = classes
print(train_df2.shape)
train_df2.head()

In [ ]:
#print('Class count before:\n'+str(train_df2['class'].value_counts()))

# Separate majority and minority classes
#train_df2_class1 = train_df2[train_df2['class']==1]
#train_df2_class2 = train_df2[train_df2['class']==2]
#train_df2_class3 = train_df2[train_df2['class']==3]
#train_df2_class4 = train_df2[train_df2['class']==4]
#train_df2_class0 = train_df2[train_df2['class']==0]

# Downsample majority class
#train_df2_class1_downsampled = resample(train_df2_class1, replace=False, n_samples=min(class_frquencies), random_state=123)
#train_df2_class3_downsampled = resample(train_df2_class3, replace=False, n_samples=min(class_frquencies), random_state=123)
#train_df2_class4_downsampled = resample(train_df2_class4, replace=False, n_samples=min(class_frquencies), random_state=123)
#train_df2_class0_downsampled = resample(train_df2_class0, replace=False, n_samples=min(class_frquencies), random_state=123)

# Combine minority class with downsampled majority class
#train_df2 = pd.concat([train_df2_class1_downsampled, train_df2_class2, train_df2_class3_downsampled, train_df2_class4_downsampled])
 
#print('Class count after:\n'+str(train_df2['class'].value_counts()))

------------------------

**Load data:**
-------------------
* helper methods
* create dataset
* visualize example images

In [ ]:
def mask2rle(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels= img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def rle2mask(rle, input_shape, label):
    width, height = input_shape[:2]
    
    mask= np.zeros( width*height ).astype(np.uint8)
    
    array = np.asarray([int(x) for x in rle.split()])
    starts = array[0::2]
    lengths = array[1::2]

    current_position = 0
    for index, start in enumerate(starts):
        mask[int(start):int(start+lengths[index])] = label
        current_position += lengths[index]
        
    return mask.reshape(height, width).T

def build_masks(rles, input_shape):
    depth = len(rles)
    masks = np.zeros(input_shape)
    
    for i, rle in enumerate(rles):
        if type(rle) is str:
            masks += rle2mask(rle, input_shape, i+1)
    
    return masks

def build_rles(masks):
    width, height, depth = masks.shape
    
    rles = [mask2rle(masks[:, :, i+1])
            for i in range(depth-1)]
    
    return rles


In [ ]:
def load_img(code, base, resize=True):
    path = f'{base}/{code}'
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    if resize:
        img = cv2.resize(img, (800, 128))
    
    return img

def validate_path(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [ ]:
#train_path = '../tmp/train'
train_path = '../input/severstal-steel-defect-detection/train_images'
#validate_path(train_path)

#for i, code in enumerate(tqdm(train_df2['ImageId'])):
#    img = load_img(
#        code,
#        base='../input/severstal-steel-defect-detection/train_images'
#    )
#    path = code.replace('.jpg', '')
#    cv2.imwrite(f'{train_path}/{path}.png', img)

In [ ]:
#train_df2['EP1'] = train_df2['EP1'].apply(
#    lambda x: x if type(x) == float else mask2rle(cv2.resize(rle2mask(x, (256, 1600), 1), (800, 128), interpolation=cv2.INTER_NEAREST)) 
#)
#train_df2['EP2'] = train_df2['EP2'].apply(
#    lambda x: x if type(x) == float else mask2rle(cv2.resize(rle2mask(x, (256, 1600), 1), (800, 128), interpolation=cv2.INTER_NEAREST)) 
#)
#train_df2['EP3'] = train_df2['EP3'].apply(
#    lambda x: x if type(x) == float else mask2rle(cv2.resize(rle2mask(x, (256, 1600), 1), (800, 128), interpolation=cv2.INTER_NEAREST)) 
#)
#train_df2['EP4'] = train_df2['EP4'].apply(
#    lambda x: x if type(x) == float else mask2rle(cv2.resize(rle2mask(x, (256, 1600), 1), (800, 128), interpolation=cv2.INTER_NEAREST)) 
#)

In [ ]:
#train_df2['ImageId'] = train_df2['ImageId'].apply(
#    lambda x: x.replace('.jpg', '.png')
#)

In [ ]:
#train_df2.head()

In [ ]:
class SteelDataset(Dataset):
    def __init__(self, df, x_col, y_cols, path):
        self.df = df
        self.x_col = x_col
        self.y_cols = y_cols
        self.path = path
        self.mean = np.array([0.485, 0.456, 0.406])
        self.std = np.array([0.229, 0.224, 0.225])
        self.edge_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7, 7))
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        # load image
        im_name = self.df[self.x_col].iloc[idx]
        img_path = f"{self.path}/{im_name}"
        img = cv2.imread(img_path)
   
        # load mask
        image_df = self.df[self.df[self.x_col] == im_name]
        rles = [image_df['EP1'].values[0],image_df['EP2'].values[0],image_df['EP3'].values[0],image_df['EP4'].values[0]]
        masks = build_masks(rles, (256, 1600)).astype(np.uint8)
        #masks = np.expand_dims(masks, axis=-1)
        
        # create borders
        rgb_masks = cv2.cvtColor(masks,cv2.COLOR_GRAY2RGB)
        rgb_masks[np.where((rgb_masks==[1,1,1]).all(axis=2))] = [0,0,128]
        rgb_masks[np.where((rgb_masks==[2,2,2]).all(axis=2))] = [0,128,0]
        rgb_masks[np.where((rgb_masks==[3,3,3]).all(axis=2))] = [0,128,128]
        rgb_masks[np.where((rgb_masks==[4,4,4]).all(axis=2))] = [128,0,0]
        cgt = cv2.Canny(rgb_masks, 5, 5, apertureSize=7)
        cgt = cv2.dilate(cgt, self.edge_kernel).astype(np.uint8)
        cgt[cgt == 255] = 1
        #cgt = np.expand_dims(cgt, axis=-1)
        
        # normalize image
        img = img.astype(np.float32) / 255.0
        img = img - self.mean
        img = img / self.std
        
        # convert to torch tensors
        img = torch.tensor(img)
        img = img.permute(2, 0, 1)
        masks = torch.tensor(masks)
        masks = masks.permute(0, 1)
        cgt = torch.tensor(cgt)
        cgt = cgt.permute(0, 1)
        
        #combine
        sample = {'image': img, 'masks': masks, 'cgt': cgt}
        return sample

In [ ]:
steel_dataset = SteelDataset(train_df2, 'ImageId', ['EP1', 'EP2', 'EP3', 'EP4'], train_path)

In [ ]:
inc = 0
fig, axs = plt.subplots(15, figsize=(25, 25))
for i in range(0,15,3):
    axs[i].imshow(steel_dataset.__getitem__(i+inc)['image'].permute(1, 2, 0)[:,:,0], cmap='gray')
    axs[i].axis('off')
    axs[i+1].imshow(steel_dataset.__getitem__(i+inc)['masks'])
    axs[i+1].axis('off')
    axs[i+2].imshow(steel_dataset.__getitem__(i+inc)['cgt'])
    axs[i+2].axis('off')

--------------------------

**Metrics:**
------------

In [ ]:
#according paper said , sigmoid better than softmax
class SigmoidFocalLoss(nn.Module):
    def __init__(self, ignore_label, gamma=2.0, alpha=0.25,
                 reduction='mean'):
        super(SigmoidFocalLoss, self).__init__()
        self.ignore_label = ignore_label
        self.gamma = gamma
        self.alpha = alpha
        self.reduction = reduction

    def forward(self, pred, target):
        b, h, w = target.size()
        pred = pred.view(b, -1, 1)
        pred_sigmoid = pred.sigmoid()
        target = target.view(b, -1).float()
        mask = (target.ne(self.ignore_label)).float()
        target = mask * target
        onehot = target.view(b, -1, 1)

        pos_part = (1 - pred_sigmoid) ** self.gamma * torch.log(pred_sigmoid + 1e-4)
        neg_part = pred_sigmoid ** self.gamma *  torch.log(1-pred_sigmoid+1e-4)

        loss = -(self.alpha * pos_part*(onehot == 1).float() + (1 - self.alpha) * neg_part*(onehot==0).float()).sum(
            dim=-1)*mask
        if self.reduction == 'mean':
            loss = loss.mean()

        return loss

In [ ]:
# taken from https://www.kaggle.com/iafoss/unet34-dice-0-87
def dice(pred, targs):
    smooth = 1.0
    dice = np.zeros(len(pred))
    class_dice = [[] for i in range(len(pred))]
    targs = torch.squeeze(targs)
    masks_target = torch.from_numpy(np.stack([np.where(targs==label, 1, 0) for label in range(5)], axis=1))
    
    for i in range(len(pred)):
        tmp_class_dice = np.zeros(5)
        for j in range(5):
            iflat = pred[i][j].view(-1).float()
            tflat = masks_target[i][j].view(-1).float()
            intersection = (iflat * tflat).sum()
            tmp_class_dice[j] += (2.0 * intersection + smooth).item()/(iflat.sum() + tflat.sum() + smooth).item()
        class_dice[i] = tmp_class_dice
    class_dice = np.sum(class_dice, axis=0) / len(pred)
    dice = np.mean(class_dice)
    return np.round(class_dice,3), dice

In [ ]:
x = torch.randint(0,5,(4,1,128,800))
x2 = torch.from_numpy(np.stack([np.where(torch.squeeze(x)==label, 1, 0) for label in range(5)], axis=1))

In [ ]:
print(dice(x2,x))

In [ ]:
# taken from https://www.kaggle.com/iafoss/unet34-dice-0-87
def IoU(pred, targs):
    pred = (pred>0).double()
    intersection = (pred*targs).sum()
    return intersection / ((pred+targs).sum() - intersection + 1.0)

--------------------

**Create model:**
-------------------

**ResNet:**

In [ ]:
import torch.nn as nn
import math
import torch.utils.model_zoo as model_zoo


__all__ = ['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101',
           'resnet152']


model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
}


def conv3x3(in_planes, out_planes, stride=1):
    "3x3 convolution with padding"
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=1000):
        self.inplanes = 64
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AvgPool2d(7)
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        h1 = self.layer1(x)
        h2 = self.layer2(h1)
        h3 = self.layer3(h2)
        h4 = self.layer4(h3)

        return [h1, h2, h3, h4]

def resnet18(pretrained=False, **kwargs):
    """Constructs a ResNet-18 model.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(BasicBlock, [2, 2, 2, 2], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet18']))
    return model

def resnet34(pretrained=False, **kwargs):
    """Constructs a ResNet-34 model.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(BasicBlock, [3, 4, 6, 3], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet34']))
    return model

def resnet50(pretrained=False, **kwargs):
    """Constructs a ResNet-50 model.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(Bottleneck, [3, 4, 6, 3], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet50']))
    return model

def resnet101(pretrained=False, **kwargs):
    """Constructs a ResNet-101 model.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(Bottleneck, [3, 4, 23, 3], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet101']))
    return model

def resnet152(pretrained=False, **kwargs):
    """Constructs a ResNet-152 model.

    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(Bottleneck, [3, 8, 36, 3], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet152']))
    return model

**DFN:**

In [ ]:
class ConvBnRelu(nn.Module):
    def __init__(self, in_planes, out_planes, ksize, stride, pad, dilation=1,
                 groups=1, has_bn=True, norm_layer=nn.BatchNorm2d, bn_eps=1e-5,
                 has_relu=True, inplace=True, has_bias=False):
        super(ConvBnRelu, self).__init__()
        self.conv = nn.Conv2d(in_planes, out_planes, kernel_size=ksize,
                              stride=stride, padding=pad,
                              dilation=dilation, groups=groups, bias=has_bias)
        self.has_bn = has_bn
        if self.has_bn:
            self.bn = norm_layer(out_planes, eps=bn_eps)
        self.has_relu = has_relu
        if self.has_relu:
            self.relu = nn.ReLU(inplace=inplace)

    def forward(self, x):
        x = self.conv(x)
        if self.has_bn:
            x = self.bn(x)
        if self.has_relu:
            x = self.relu(x)

        return x
    
class ChannelAttention(nn.Module):
    def __init__(self, in_planes, out_planes, reduction):
        super(ChannelAttention, self).__init__()
        self.channel_attention = SELayer(in_planes, out_planes, reduction)

    def forward(self, x1, x2):
        fm = torch.cat([x1, x2], 1)
        channel_attetion = self.channel_attention(fm)
        fm = x1 * channel_attetion + x2

        return fm

#RRB  fine tuning : actually the method only change the fcn backbone vgg to resnet ,so U need add some residual block.
#Is easily understand.
class RefineResidual(nn.Module):
    def __init__(self, in_planes, out_planes, ksize, has_bias=False,
                 has_relu=False, norm_layer=nn.BatchNorm2d, bn_eps=1e-5):
        super(RefineResidual, self).__init__()
        self.conv_1x1 = nn.Conv2d(in_planes, out_planes, kernel_size=1,
                                  stride=1, padding=0, dilation=1,
                                  bias=has_bias)
        self.cbr = ConvBnRelu(out_planes, out_planes, ksize, 1,
                              ksize // 2, has_bias=has_bias,
                              norm_layer=norm_layer, bn_eps=bn_eps)
        self.conv_refine = nn.Conv2d(out_planes, out_planes, kernel_size=ksize,
                                     stride=1, padding=ksize // 2, dilation=1,
                                     bias=has_bias)
        self.has_relu = has_relu
        if self.has_relu:
            self.relu = nn.ReLU(inplace=False)

    def forward(self, x):
        x = self.conv_1x1(x)
        t = self.cbr(x)
        t = self.conv_refine(t)
        if self.has_relu:
            return self.relu(t + x)
        return t + x
    
class SELayer(nn.Module):
    def __init__(self, in_planes, out_planes, reduction=16):
        super(SELayer, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(in_planes, out_planes // reduction),
            nn.ReLU(inplace=True),
            nn.Linear(out_planes // reduction, out_planes),
            nn.Sigmoid()
        )
        self.out_planes = out_planes

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, self.out_planes, 1, 1)
        return y

class DFN(nn.Module):
    def __init__(self, out_planes, criterion, aux_criterion, alpha, norm_layer=nn.BatchNorm2d):
        super(DFN, self).__init__()
        self.backbone = resnet101(pretrained=True)
        #for param in self.backbone.parameters():   # Freeze resnet layers
        #    param.requires_grad = False
        self.business_layer = []

        smooth_inner_channel = 512
        self.global_context = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            ConvBnRelu(2048, smooth_inner_channel, 1, 1, 0,
                       has_bn=True,
                       has_relu=True, has_bias=False, norm_layer=norm_layer)
        )
        self.business_layer.append(self.global_context)

        stage = [2048, 1024, 512, 256]
        self.smooth_pre_rrbs = []
        self.cabs = []
        self.smooth_aft_rrbs = []
        self.smooth_heads = []

        #every stage we have dfn layer output?
        # top to bottom 2048 -> 256
        for i, channel in enumerate(stage):
            self.smooth_pre_rrbs.append(
                RefineResidual(channel, smooth_inner_channel, 3, has_bias=False,
                               has_relu=True, norm_layer=norm_layer))
            self.cabs.append(
                ChannelAttention(smooth_inner_channel * 2,
                                 smooth_inner_channel, 1))
            self.smooth_aft_rrbs.append(
                RefineResidual(smooth_inner_channel, smooth_inner_channel, 3,
                               has_bias=False,
                               has_relu=True, norm_layer=norm_layer))
            self.smooth_heads.append(
                DFNHead(smooth_inner_channel, out_planes, scale=2 ** (5 - i),
                        norm_layer=norm_layer))

        stage.reverse()
        border_inner_channel = 5
        self.border_pre_rrbs = []
        self.border_aft_rrbs = []
        self.border_heads = []


        for i, channel in enumerate(stage):
            self.border_pre_rrbs.append(
                RefineResidual(channel, border_inner_channel, 3, has_bias=False,
                               has_relu=True, norm_layer=norm_layer))
            self.border_aft_rrbs.append(
                RefineResidual(border_inner_channel, border_inner_channel, 3,
                               has_bias=False,
                               has_relu=True, norm_layer=norm_layer))
            self.border_heads.append(
                DFNHead(border_inner_channel, 1, 4, norm_layer=norm_layer))

        self.smooth_pre_rrbs = nn.ModuleList(self.smooth_pre_rrbs)
        self.cabs = nn.ModuleList(self.cabs)
        self.smooth_aft_rrbs = nn.ModuleList(self.smooth_aft_rrbs)
        self.smooth_heads = nn.ModuleList(self.smooth_heads)
        self.border_pre_rrbs = nn.ModuleList(self.border_pre_rrbs)
        self.border_aft_rrbs = nn.ModuleList(self.border_aft_rrbs)
        self.border_heads = nn.ModuleList(self.border_heads)

        #smooth model
        self.business_layer.append(self.smooth_pre_rrbs)
        self.business_layer.append(self.cabs)
        self.business_layer.append(self.smooth_aft_rrbs)
        self.business_layer.append(self.smooth_heads)

        #border_layer model
        self.business_layer.append(self.border_pre_rrbs)
        self.business_layer.append(self.border_aft_rrbs)
        self.business_layer.append(self.border_heads)

        self.criterion = criterion
        self.aux_criterion = aux_criterion
        self.alpha = alpha

    def forward(self, data, label=None, aux_label=None):
        blocks = self.backbone(data)


        '''
        >> Block we have 4 conv1 conv2 conv3 conv4
        conv1 -> 256 1/4
        conv2 -> 512 1/8
        conv3 ->1024 1/16
        conv4 ->2048 1/32
        '''
        blocks.reverse()

        #understanding the global context meaning:

        global_context = self.global_context(blocks[0])
        #global: ->(bs,512,1,1)
        #equal to squeeze bar
        global_context = F.interpolate(global_context,
                                       size=blocks[0].size()[2:],
                                       mode='bilinear', align_corners=True)
        #this part using for smooth model ,this part sometime is important, U must carefully

        last_fm = global_context
        #last_fm --->(bs,512,1/32,1/32)
        pred_out = []

        for i, (fm, pre_rrb,
                cab, aft_rrb, head) in enumerate(zip(blocks,
                                                     self.smooth_pre_rrbs,
                                                     self.cabs,
                                                     self.smooth_aft_rrbs,
                                                     self.smooth_heads)):
            #step RRB model
            fm = pre_rrb(fm)
            #CAB para: low_level,hight_level
            fm = cab(fm, last_fm)
            fm = aft_rrb(fm)
            #RRB model ,next using to next low level , we must up sample

            #what is head mean?
            pred_out.append(head(fm))
            if i != 3:
                last_fm = F.interpolate(fm, scale_factor=2, mode='bilinear',
                                        align_corners=True)


        #every conv# have a predict label (bs,num_label,input_size,input_size)
        blocks.reverse()
        #change blocks from bottom to top
        last_fm = None
        boder_out = []
        #struct like follow border
        '''
        conv1 ---RRB------------>head
                       |
        conv2 ---RRB---+--RRB--->head
                            |
        conv3 ---RRB---+--RRB--->head
                            |
        conv4 ---RRB---+--RRB--->head
        '''
        for i, (fm, pre_rrb,
                aft_rrb, head) in enumerate(zip(blocks,
                                                self.border_pre_rrbs,
                                                self.border_aft_rrbs,
                                                self.border_heads)):
            fm = pre_rrb(fm)
            if last_fm is not None:
                fm = F.interpolate(fm, scale_factor=2 ** i, mode='bilinear',
                                   align_corners=True)
                last_fm = last_fm + fm
                last_fm = aft_rrb(last_fm)

            else:
                last_fm = fm
            boder_out.append(head(last_fm))

        #if train: loss have 4 layer
        #else print the smooth_layer.
        #this method is good , we must learning this step
        if label is not None and aux_label is not None:
            loss0 = self.criterion(pred_out[0], label)
            loss1 = self.criterion(pred_out[1], label)
            loss2 = self.criterion(pred_out[2], label)
            loss3 = self.criterion(pred_out[3], label)

            aux_loss0 = self.aux_criterion(boder_out[0], aux_label)
            aux_loss1 = self.aux_criterion(boder_out[1], aux_label)
            aux_loss2 = self.aux_criterion(boder_out[2], aux_label)
            aux_loss3 = self.aux_criterion(boder_out[3], aux_label)

            loss = loss0 + loss1 + loss2 + loss3
            aux_loss = aux_loss0 + aux_loss1 + aux_loss2 + aux_loss3
            return loss + self.alpha * aux_loss, F.log_softmax(pred_out[-1], dim=1)

        return F.log_softmax(pred_out[-1], dim=1)


class DFNHead(nn.Module):
    def __init__(self, in_planes, out_planes, scale, norm_layer=nn.BatchNorm2d):
        #remeber the scale means  upsample times
        super(DFNHead, self).__init__()
        self.rrb = RefineResidual(in_planes, out_planes * 9, 3, has_bias=False,
                                  has_relu=False, norm_layer=norm_layer)
        self.conv = nn.Conv2d(out_planes * 9, out_planes, kernel_size=1,
                              stride=1, padding=0)
        self.scale = scale

    def forward(self, x):
        x = self.rrb(x)
        x = self.conv(x)
        x = F.interpolate(x, scale_factor=self.scale, mode='bilinear',
                          align_corners=True)

        return x

-------------------

**Train:**
--------------

In [ ]:
def __init_weight(feature, conv_init, norm_layer, bn_eps, bn_momentum,
                  **kwargs):
    for name, m in feature.named_modules():
        if isinstance(m, (nn.Conv2d, nn.Conv3d)):
            conv_init(m.weight, **kwargs)
        elif isinstance(m, norm_layer):
            m.eps = bn_eps
            m.momentum = bn_momentum
            nn.init.constant_(m.weight, 1)
            nn.init.constant_(m.bias, 0)


def init_weight(module_list, conv_init, norm_layer, bn_eps, bn_momentum,
                **kwargs):
    if isinstance(module_list, list):
        for feature in module_list:
            __init_weight(feature, conv_init, norm_layer, bn_eps, bn_momentum,
                          **kwargs)
    else:
        __init_weight(module_list, conv_init, norm_layer, bn_eps, bn_momentum,
                      **kwargs)


def group_weight(weight_group, module, norm_layer, lr):
    group_decay = []
    group_no_decay = []
    for m in module.modules():
        if isinstance(m, nn.Linear):
            group_decay.append(m.weight)
            if m.bias is not None:
                group_no_decay.append(m.bias)
        elif isinstance(m, (nn.Conv2d, nn.Conv3d)):
            group_decay.append(m.weight)
            if m.bias is not None:
                group_no_decay.append(m.bias)
        elif isinstance(m, norm_layer) or isinstance(m, nn.GroupNorm):
            if m.weight is not None:
                group_no_decay.append(m.weight)
            if m.bias is not None:
                group_no_decay.append(m.bias)

    assert len(list(module.parameters())) == len(group_decay) + len(
        group_no_decay)
    weight_group.append(dict(params=group_decay, lr=lr))
    weight_group.append(dict(params=group_no_decay, weight_decay=.0, lr=lr))
    return weight_group

In [ ]:
from abc import ABCMeta, abstractmethod


class BaseLR():
    __metaclass__ = ABCMeta

    @abstractmethod
    def get_lr(self, cur_iter): pass

    
class PolyLR(BaseLR):
    def __init__(self, start_lr, lr_power, total_iters):
        self.start_lr = start_lr
        self.lr_power = lr_power
        self.total_iters = total_iters + 0.0

    def get_lr(self, cur_iter):
        return self.start_lr * (
                (1 - float(cur_iter) / self.total_iters) ** self.lr_power)

In [ ]:
# Parameters
params_train = {'batch_size': 4,
          'shuffle': True,
          'num_workers': 6}

params_val = {'batch_size': 4,
          'shuffle': False,
          'num_workers': 6}

max_epochs = 80

train, test = train_test_split(train_df2, test_size=0.15, random_state=42, stratify=train_df2['class'])

train_dataset = SteelDataset(train, 'ImageId', ['EP1', 'EP2', 'EP3', 'EP4'], train_path)
training_generator = DataLoader(train_dataset, **params_train)

val_dataset = SteelDataset(test, 'ImageId', ['EP1', 'EP2', 'EP3', 'EP4'], train_path)
validation_generator = DataLoader(val_dataset, **params_val)

In [ ]:
cudnn.benchmark = True

# config network and criterion
criterion = nn.CrossEntropyLoss(reduction='mean',
                                ignore_index=255)
aux_criterion = SigmoidFocalLoss(ignore_label=255, gamma=2.0, alpha=0.1)

model = DFN(5, criterion=criterion,
            aux_criterion=aux_criterion, alpha=0.25,
            norm_layer=nn.BatchNorm2d)
init_weight(model.business_layer, nn.init.kaiming_normal_,
            nn.BatchNorm2d, 1e-5, 0.1,
            mode='fan_in', nonlinearity='relu')

# group weight and config optimizer
base_lr = 8e-4

params_list = []
params_list = group_weight(params_list, model.backbone,
                           nn.BatchNorm2d, base_lr)
for module in model.business_layer:
    params_list = group_weight(params_list, module, nn.BatchNorm2d,
                               base_lr * 10)

optimizer = torch.optim.SGD(params_list,
                            lr=base_lr,
                            momentum=0.9,
                            weight_decay=1e-4)

# config lr policy
total_iteration = max_epochs * len(training_generator)
lr_policy = PolyLR(base_lr, 0.9, total_iteration)

In [ ]:
def save_checkpoint(state, is_best, filename='./model5.pth'):
    # taken from https://blog.floydhub.com/checkpointing-tutorial-for-tensorflow-keras-and-pytorch/
    """Save checkpoint if a new best is achieved"""
    if is_best:
        print ("=> Saving a new best")
        torch.save(state, filename)  # save checkpoint
    else:
        print ("=> Validation Accuracy did not improve")

In [ ]:
def unique(tensor1d):
    t, idx = np.unique(tensor1d.numpy(), return_inverse=True)
    return torch.from_numpy(t), torch.from_numpy(idx) 

In [ ]:
#ckpt_path = "../input/severstal-dfn/model.pth"
#state = torch.load(ckpt_path, map_location=lambda storage, loc: storage)
#model.load_state_dict(state["state_dict"])
model.to(device)

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
# tensorboard
writer = SummaryWriter()

best_loss_set = False
model.train()
for epoch in range(max_epochs):
    running_loss_train = 0
    running_dice_coeff_train = 0
    running_iou_train = 0
    #start = time.strftime("%H:%M:%S")
    
    for idx, minibatch in enumerate(training_generator):
        optimizer.zero_grad()

        imgs = minibatch['image'].float()
        gts = minibatch['masks'].long()
        cgts = minibatch['cgt'].float()

        imgs = imgs.cuda()
        gts = gts.cuda()
        cgts = cgts.cuda()
        
        #print(unique(imgs.cpu().detach()))
        #print(unique(gts.cpu().detach()))
        #print(unique(cgts.cpu().detach()))


        loss, outputs = model(imgs, gts, cgts)

        #print(unique(outputs.cpu().detach()))
        #y = gts.cpu().detach()
        
        # Print metrics
        running_loss_train += loss.item()
        _, dice_coeff = dice(torch.exp(outputs.double().cpu().detach()).int(), gts.double().cpu().detach())
        running_dice_coeff_train += dice_coeff
        #iou = IoU(outputs.double(), gts.double())
        #running_iou_train += iou

        current_idx = epoch * len(training_generator) + idx
        lr = lr_policy.get_lr(current_idx)

        optimizer.param_groups[0]['lr'] = lr
        optimizer.param_groups[1]['lr'] = lr
        for i in range(2, len(optimizer.param_groups)):
            optimizer.param_groups[i]['lr'] = lr * 10

        loss.backward()
        optimizer.step()
        
        print("Batch {}/{}, Loss: {:.5f}, Dice-Coeff: {:.5f}".format(idx+1, len(training_generator),loss,dice_coeff), end='\r')
        sys.stdout.flush()    

        
    with torch.set_grad_enabled(False):
        running_loss_val = 0
        running_dice_coeff_val = 0
        running_iou_val = 0
        running_class_dice_coeff_val = np.zeros(5)
        for idx, minibatch in enumerate(validation_generator):
            imgs = minibatch['image'].float()
            gts = minibatch['masks'].long()
            cgts = minibatch['cgt'].float()

            imgs = imgs.cuda(non_blocking=True)
            gts = gts.cuda(non_blocking=True)
            cgts = cgts.cuda(non_blocking=True)

            loss, outputs = model(imgs, gts, cgts)

            running_loss_val += loss.item()
            class_dice_coeff, dice_coeff = dice(torch.exp(outputs.double().cpu()).int(), gts.double().cpu())
            running_dice_coeff_val += dice_coeff
            running_class_dice_coeff_val = np.add(running_class_dice_coeff_val, class_dice_coeff)
            
            #iou = IoU(outputs.double(), gts.double())
            #running_iou_val += iou
        loss = running_loss_train/len(training_generator)
        val_loss = running_loss_val/len(validation_generator)
        dice_coeff = running_dice_coeff_train/len(training_generator)
        val_dice_coeff = running_dice_coeff_val/len(validation_generator)
        running_class_dice_coeff_val /= len(validation_generator)
        
                    
        writer.add_scalar('Loss', loss, epoch)
        writer.add_scalar('Val-Loss', val_loss, epoch)
        writer.add_scalar('Dice-Coeff', dice_coeff, epoch)
        writer.add_scalar('Val-Dice-Coeff', val_dice_coeff, epoch)
        writer.add_scalar('Val-Class0-Dice-Coeff', running_class_dice_coeff_val[0], epoch)
        writer.add_scalar('Val-Class1-Dice-Coeff', running_class_dice_coeff_val[1], epoch)
        writer.add_scalar('Val-Class2-Dice-Coeff', running_class_dice_coeff_val[2], epoch)
        writer.add_scalar('Val-Class3-Dice-Coeff', running_class_dice_coeff_val[3], epoch)
        writer.add_scalar('Val-Class4-Dice-Coeff', running_class_dice_coeff_val[4], epoch)
        #iou = running_iou_train/len(training_generator)
        #val_iou = running_iou_val/len(validation_generator)
        print("Epoch {}/{}, Loss: {:.3f}, Val-Loss: {:.3f}, Dice-Coeff: {:.3f}, Val-Dice-Coeff: {:.3f}, Val-Class-Dice-Coeff: {}".format(epoch+1, max_epochs, loss, val_loss, dice_coeff, val_dice_coeff, running_class_dice_coeff_val))

        # Save checkpoint if is a new best
        if best_loss_set:
            is_best = bool(best_loss < val_dice_coeff)
            best_loss = max(val_dice_coeff, best_loss)
            save_checkpoint({
                'epoch': epoch + 1,
                'state_dict': model.state_dict(),
                'best_loss': best_loss
            }, is_best)
        else:
            best_loss = val_dice_coeff
            save_checkpoint({
                'epoch': epoch + 1,
                'state_dict': model.state_dict(),
                'best_loss': best_loss
            }, True)
            best_loss_set = True

        # Reduce LR on Plateau
        #scheduler.step(loss)
writer.close()

In [ ]:
# freeze resnet layers -
# changed SGD to Adam

In [ ]:
ckpt_path = "./model.pth"
state = torch.load(ckpt_path, map_location=lambda storage, loc: storage)
model.load_state_dict(state["state_dict"])
model.to(device)

In [ ]:
preds = []
with torch.set_grad_enabled(False):
    for idx, minibatch in enumerate(validation_generator):
        imgs = minibatch['image'].float()#

        imgs = imgs.cuda(non_blocking=True)#

        outputs = model(imgs)
        masks = torch.exp(outputs.double().cpu()).int()
        for mask in masks:
            preds.append(build_rles(mask.permute(1,2,0).numpy()))

In [ ]:
for i in range(200,400,5):
    inc = i
    fig, axs = plt.subplots(15, figsize=(25, 25))
    for i in range(0,15,3):
        masks = build_masks(preds[i+inc], (128, 800))
        axs[i].imshow(val_dataset[i+inc]['image'].permute(1, 2, 0)[:,:,0], cmap='gray')
        axs[i].axis('off')
        axs[i+1].imshow(masks)
        axs[i+1].axis('off')
        axs[i+2].imshow(val_dataset[i+inc]['masks'])
        axs[i+2].axis('off')
    plt.show()

**Threshold:**
------------

In [ ]:
#def post_process(mask_in, min_size=3500):
#    '''Post processing of each predicted mask, components with lesser number of pixels
#    than `min_size` are ignored'''
#    mask = cv2.threshold(mask_in, 0.5, 1, cv2.THRESH_BINARY)[1]
#    num_component, component = cv2.connectedComponents(mask.astype(np.uint8))
#    predictions = np.zeros((256, 1600), np.float32)
#    num = 0
#    for c in range(1, num_component):
#        p = (component == c)
#        if p.sum() > min_size:
#            predictions[p] = 1
#            num += 1
#    mask_out = np.multiply(mask_in, predictions)
#    #print(mask_out.shape)
#    return mask_out, num

In [ ]:
#preds = []
#with torch.set_grad_enabled(False):
#    dice_coeff_val = []
#    running_dice_coeff_val = np.zeros(11)
#    for idx, minibatch in enumerate(tqdm(validation_generator)):
#        imgs = minibatch['image'].float()
#        gts = minibatch['masks'].int()#

#        imgs = imgs.cuda(non_blocking=True)
#
#        outputs = model(imgs)
#        masks = torch.exp(outputs.double().cpu())
#        l = 0
#        for k in range(0,5001,500):
#            for i, masks_ in enumerate(masks):
#                tmp_masks = []
#                for j, mask in enumerate(masks_):
#                    pred, num = post_process(mask.numpy(), k)
#                    tmp_masks.append(pred)
#                masks[i] = torch.Tensor(tmp_masks).double()
#                #preds.append(build_rles(pred.permute(1,2,0).numpy()))#

#            dice_coeff = dice(masks.int(), gts)
#            running_dice_coeff_val[l] += dice_coeff
#            l += 1
#    dice_coeff_val = [running_dice_coeff_val[o]/len(validation_generator) for o in range(len(running_dice_coeff_val))]
#    print(dice_coeff_val)

In [ ]:
#plt.plot(list(range(0,5001,500)), dice_coeff_val)

In [ ]:
print(lr)

In [ ]:
from IPython.display import FileLink
FileLink('./model.pth')